In [ ]:
note = "ccs"
allname = [
    "customized-" + x + "_moderate25"
    for x in ["1", "2", "3", "4", "5", "front", "tamaraw", "regulator", "wfgan"]
]
allname.append("empty")

In [ ]:
tasks = []
for train in allname:
    for test in allname:
        tasks.append(f"python evaluate.py -n {note} --train {train} --test {test}")
        # tasks.append(f"python evaluate.py --attack DF -n {note} --train {train} --test {test}")
        # tasks.append(f"python evaluateow.py -n {note}ow --train {train} --test {test}")
        # tasks.append(f"python evaluateow.py --attack DF -n {note}ow --train {train} --test {test}")

In [ ]:
tasks = []
tt = []
for i in range(1, 6):
    train = "empty customized-1 customized-2 customized-3 customized-4 customized-5".split()
    test = train.pop(i)
    train = "&".join(train)
    tt.append((train, test))

for i in range(10):
    test = "customized-1 customized-2 customized-3 customized-4 customized-5 empty customized-front customized-tamaraw customized-wfgan customized-regulator".split()[
        i
    ]
    train = (
        "empty customized-front customized-tamaraw customized-wfgan customized-regulator".replace(
            " ", "&"
        )
    )
    tt.append((train, test))
for train, test in tt:
    tasks.append(f'python evaluateow.py -n {note}  --train "{train}" --test {test}')

In [ ]:
avail_gpu = [2, 3, 4, 5, 6, 7]
task_per_gpu = 5
for i, task in enumerate(tasks):
    print(task + f" -g {avail_gpu[i % len(avail_gpu)]} &")
    if (i + 1) % (task_per_gpu * len(avail_gpu)) == 0:
        print("wait")
if len(tasks) % (task_per_gpu * len(avail_gpu)) != 0:
    print("wait")

python evaluate.py -n 20240330middle  --train "empty&customized-front&customized-tamaraw&customized-wfgan&customized-regulator&customized-1&customized-2&customized-3&customized-4&customized-5" --test customized-6 -g 2
python evaluate.py -n 20240330middle  --train "empty&customized-front&customized-tamaraw&customized-wfgan&customized-regulator&customized-1&customized-2&customized-3&customized-4&customized-5" --test customized-7 -g 2
python evaluate.py -n 20240330middle  --train "empty&customized-front&customized-tamaraw&customized-wfgan&customized-regulator&customized-1&customized-2&customized-3&customized-4&customized-5" --test customized-8 -g 2
wait

In [ ]:
from pathlib import Path
import pandas as pd
import dataset
import json
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict


def get_overhead(name, defense):
    ds = dataset.get_ds(name, use_cache=True)
    return ds.summary_overhead(defense)


run_test_dir = Path("run/test/ccs")
for ds_dir in run_test_dir.iterdir():
    ds = ds_dir.name
    for attack_dir in ds_dir.iterdir():
        data = defaultdict(lambda: defaultdict(dict))
        attack = attack_dir.name
        for train_dir in attack_dir.iterdir():
            train = train_dir.name
            for test_file in train_dir.glob("*.log"):
                test = test_file.stem
                overhead = get_overhead(ds, test)
                data["overhead-time"][test] = overhead[0]
                data["overhead-data"][test] = overhead[1]
                result = eval(test_file.read_text())
                data[train][test] = result["accuracy"]
        data = pd.DataFrame(data).T
        data.index = [x.replace("customized-", "") for x in data.index]
        data.index = [x.replace("empty&", "") for x in data.index]
        data = data.reindex(
            sorted(
                data.index,
                key=lambda x: (
                    "overhead" not in x,
                    "empty" != x,
                    not x[0].isdigit(),
                    len(x),
                    x,
                ),
            )
        )
        data = data.reindex(
            sorted(data.columns, key=lambda x: (len(x), x)),
            axis=1,
        )
        data.to_csv(f"log/middle-{ds}-{attack}.csv")
        plt.figure(figsize=(8, 8))
        ax = sns.heatmap(data.round(4), annot=True, cmap="YlGnBu")
        ax.set_xlabel("Test")
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        ax.xaxis.set_tick_params(rotation=45)
        ax.set_ylabel("Train")
        ax.yaxis.set_tick_params(rotation=0)
        plt.tight_layout()
        ax.set_title(f"{ds} {attack}")
        plt.show()

In [ ]:
run_test_dir = Path("run/test/20240330middle")
ds = "ours"
attack = "RF"
attack_dir = run_test_dir / ds / attack

data = defaultdict(lambda: defaultdict(dict))
for train_dir in attack_dir.iterdir():
    train = train_dir.name
    for test_file in train_dir.glob("*.log"):

        test = test_file.stem
        if train[-1] not in "12345 " or test[-1] not in "12345":
            continue
        overhead = get_overhead(ds, test)
        data["overhead-time"][test] = overhead[0]
        data["overhead-data"][test] = overhead[1]
        result = eval(test_file.read_text())

        data[train][test] = result["accuracy"]
data = pd.DataFrame(data).T
data.index = [x.replace("customized-", "") for x in data.index]
data.index = [x.replace("empty&", "") for x in data.index]
data = data.reindex(
    sorted(
        data.index,
        key=lambda x: (
            "overhead" not in x,
            "empty" != x,
            not x[0].isdigit(),
            len(x),
            x,
        ),
    )
)
data = data.reindex(
    sorted(data.columns, key=lambda x: (len(x), x)),
    axis=1,
)
data.to_csv(f"log/middle-{ds}-{attack}.csv")
plt.figure(figsize=(10, 10))
ax = sns.heatmap(data.round(2), annot=True, cmap="YlGnBu")
ax.set_xlabel("Test")
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
ax.xaxis.set_tick_params(rotation=45)
ax.set_ylabel("Train")
ax.yaxis.set_tick_params(rotation=0)
plt.tight_layout()
ax.set_title(f"{ds} {attack}")
plt.show()